In [67]:
import sys
sys.path.append('../../')
import os

import pandas as pd
from utils import fill_empty, VColumns
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load input file

In [68]:
df1 = pd.read_csv("v1-start.csv")

# Start elaborating columns

Create a new dataframe with empty values, so that we can start filling in values from the input dataframe, without overriding columns.

In [69]:
# drop lines with empty name1, name2, winery
df1 = df1[df1['name1'].notna() & df1['name2'].notna() & df1['winery'].notna()]

# drop lines with qty <= 0
df1 = df1[df1['qty'] > 0]

df1.head()

,type,size,storage_area,name1,name2,winery,info,vintage,qty,purchase_price_eur,sales_price_eur
0,WHITE,BOTTLE,D1,Bianco,Kaplja,Damijan Podversic,NaN,2018.0,12.0,28.50 €,64.00 €
1,WHITE,BOTTLE,D1,Bianco,Kaplja,Damijan Podversic,NaN,2017.0,4.0,25.00 €,64.00 €
2,WHITE,BOTTLE,B5,Bianco,Pulsar,Roberto Ferrari,NaN,2018.0,7.0,26.50 €,68.00 €
3,WHITE,BOTTLE,NaN,Bianco,Pulsar,Roberto Ferrari,NaN,2017.0,2.0,26.50 €,68.00 €
4,WHITE,BOTTLE,NaN,Bianco,Pulsar,Roberto Ferrari,NaN,2016.0,2.0,26.50 €,72.00 €


In [70]:
# create new empty dataframe
df = pd.DataFrame(columns=VColumns.v2())
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible


In [71]:
# copy values from original columns to new columns
df['external_id'] = None
df['type'] = df1['type']
df['name'] = df1['name1'] + ' ' + df1['name2']
df['winery_name'] = df1['winery']
df["size"] = df1["size"]
df["vintage"] = df1["vintage"]
df["quantity"] = df1["qty"]
df["storage_area"] = df1["storage_area"]
df["info"] = df1["info"]
df["internal_notes"] = None
df["visible"] = True

df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,NaN,WHITE,Bianco Kaplja,Damijan Podversic,NaN,BOTTLE,2018.0,NaN,NaN,12.0,D1,None,True
1,NaN,WHITE,Bianco Kaplja,Damijan Podversic,NaN,BOTTLE,2017.0,NaN,NaN,4.0,D1,None,True
2,NaN,WHITE,Bianco Pulsar,Roberto Ferrari,NaN,BOTTLE,2018.0,NaN,NaN,7.0,B5,None,True
3,NaN,WHITE,Bianco Pulsar,Roberto Ferrari,NaN,BOTTLE,2017.0,NaN,NaN,2.0,NaN,None,True
4,NaN,WHITE,Bianco Pulsar,Roberto Ferrari,NaN,BOTTLE,2016.0,NaN,NaN,2.0,NaN,None,True


In [72]:
df1['sales_price_eur'] = df1['sales_price_eur'].fillna(0)
df['price'] = df1['sales_price_eur'].apply(lambda x: int(float(str(x).replace('€', '').replace(',', ''))*100)) # convert to cents

df['purchase_price_eur'] = df1['purchase_price_eur'].fillna(0)
df['purchase_price'] = df1['purchase_price_eur'].apply(lambda x: int(float(str(x).replace('€', '').replace(',', ''))*100)) # convert to cents

df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible,purchase_price_eur
0,NaN,WHITE,Bianco Kaplja,Damijan Podversic,NaN,BOTTLE,2018.0,6400,2850,12.0,D1,None,True,28.50 €
1,NaN,WHITE,Bianco Kaplja,Damijan Podversic,NaN,BOTTLE,2017.0,6400,2500,4.0,D1,None,True,25.00 €
2,NaN,WHITE,Bianco Pulsar,Roberto Ferrari,NaN,BOTTLE,2018.0,6800,2650,7.0,B5,None,True,26.50 €
3,NaN,WHITE,Bianco Pulsar,Roberto Ferrari,NaN,BOTTLE,2017.0,6800,2650,2.0,NaN,None,True,26.50 €
4,NaN,WHITE,Bianco Pulsar,Roberto Ferrari,NaN,BOTTLE,2016.0,7200,2650,2.0,NaN,None,True,26.50 €


In [73]:
df = fill_empty(df, VColumns.v2(), False)
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible,purchase_price_eur
0,,WHITE,Bianco Kaplja,Damijan Podversic,,BOTTLE,2018,6400,2850,12,D1,,True,28.50 €
1,,WHITE,Bianco Kaplja,Damijan Podversic,,BOTTLE,2017,6400,2500,4,D1,,True,25.00 €
2,,WHITE,Bianco Pulsar,Roberto Ferrari,,BOTTLE,2018,6800,2650,7,B5,,True,26.50 €
3,,WHITE,Bianco Pulsar,Roberto Ferrari,,BOTTLE,2017,6800,2650,2,,,True,26.50 €
4,,WHITE,Bianco Pulsar,Roberto Ferrari,,BOTTLE,2016,7200,2650,2,,,True,26.50 €


# Merge files and write output

If there are multiple files, merge them into one and create single output file.

In [74]:
df.head()

df_out = pd.concat([df], ignore_index=True)
df_out.to_csv("v2-cleaned.csv", index=False)

bool()

False